# 01 - Pré-processamento

Pipeline padronizado de preparação dos dados.

In [1]:
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer


In [2]:
df = pd.read_csv('../data/tratados/amostra_19-21.csv', sep= ',', encoding="ISO-8859-1")

In [4]:

X = df.drop('gravidade', axis=1)
y = df['gravidade']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

categorical_features = ['br', 'frota',
    'uf', 'km', 'municipio', 'dia_semana', 'fase_dia',
    'sentido_via', 'condicao_metereologica', 'tipo_pista',
    'tracado_via', 'uso_solo', 'tipo_veiculo'
]

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocess = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features)
    ]
)


In [5]:
# Salvar o pipeline de pré-processamento
joblib.dump(preprocess, 'preprocess.pkl')

# (opcional, mas recomendado) salvar os splits
joblib.dump(
    (X_train, X_test, y_train, y_test),
    'data_splits.pkl'
)

print("Pré-processamento e splits salvos com sucesso.")


Pré-processamento e splits salvos com sucesso.
